In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../notebooks_publications/')
import bluranalysis as analysis

# plt.style.use('deblur')

# DNF and SNR Bounds

For any illumination sequence with exactly $N$ pulses (or more generally, with total illumination $\sum_{i=1}^n v_i = N)$, we will have (assuming the illumination power of a single pulse is normalized to $1$) a signal-to-noise ratio (SNR) of
$$SNR = \frac{\bar{i}_0 N}{f \sqrt{\sigma_{\text{gray}}^2 + \sigma_d^2 N}}\:,$$
where $\bar{i}_0$ is the average intensity of the object, $\sigma_{\text{gray}}$ is the signal-independent part of the noise, and $\sigma_d$ is the weight on the signal-dependent part of the noise. We define the (normalized) deconvolution noise factor (DNF) $f$ as
$$f = \sqrt{\frac{1}{m} \sum_{i=1}^m \frac{\sigma_1^2}{\sigma_i^2}}\:,$$
where $\sigma_1,...,\sigma_m$ are the ordered singular values of the blur operation $A$. Since $A$ is a convolutional operator with kernel $b=\sum_{j=1}^n v_j \delta_j$, the singular values are given by the power spectrum $\{|(Fb)_i|^2\}_{i=1}^m$. The $v_i$ are illumination values and the $\delta_i$ are the positions.

In this notebook, I discuss

1. a *lower bound* on the DNF, which translates to an *upper bound* on the SNR, i.e. the *best* we could ever do.
2. an *upper bound* on the DNF, which translates into a *lower bound* on the SNR, i.e. the *worst* we can expect to do. This is somewhat less important to our story.

# Lower Bound on DNF

Lemma: We have that
$ f^2 \geq \frac{N^2}{\frac{1}{m}\sum_{i=1}^m \sigma_i^2}$.

*Proof*: Starting with $f^2 = \sigma_1^2\cdot \frac{1}{m} \sum_{i=1}^m \frac{1}{\sigma_i^2}$, note that $\frac{1}{m} \sum_{i=1}^m \frac{1}{\sigma_i^2}$ is the reciporocal of the *harmonic mean* of $\{\sigma_1^2,...,\sigma_m^2\}$. Since the harmonic mean is always less than the *arithmetic mean* [1], we have that
$$ \frac{1}{m} \sum_{i=1}^m \frac{1}{\sigma_i^2}\geq \frac{1}{\frac{1}{m}\sum_{i=1}^m \sigma_i^2}\:. $$
Then the result follows by the observation that $\sigma_1$ is the DC component of the signal, which is $\sum_{i=1}^n v_i = N$.


[1] e.g. https://sites.math.washington.edu/~dumitriu/Inequalities 

## From Power Spectrum Average to Illumination Power (Parseval's)

Lemma: We have the following relationship between average squared singular value and illumination values, $\frac{1}{m}\sum_{i=1}^m \sigma_i^2 = \sum_{j=1}^n v_j^2$.

*Proof*: Since the singular values are given by the vector $Fb = F\sum_{j=1}^n v_j \delta_j$, the average squared singular values is given by the inner product
$$\frac{1}{m} \sum_{i=1}^m \sigma_i^2 = \frac{1}{m} (Fb)^H Fb =  \frac{1}{m} \sum_{j=1}^n \sum_{\ell=1}^n v_jv_\ell \delta_j^H F^H F\delta_\ell =  \sum_{j=1}^n  v_j^2 $$
where the final simplification comes from noting that $F^H F = mI$ (since we use un-normalized DFT matrices) and $\delta_j^H\delta_\ell = \mathbf{1}\{j=\ell\}$.

TODO: should we be more careful about DFT scaling in various definitions?

In [5]:
# A numerical validation of the above lemma

def check_equiv(n_pulses, kernel_length, padding_size=0, generation='binary'):
    kernel = np.zeros(kernel_length+padding_size)
    if generation == 'binary':
        indicies = np.random.choice(kernel_length, size=n_pulses, replace=False)
        kernel[indicies] = 1.0
    else:
        kernel[0:kernel_length] = np.random.uniform(0,1, size=kernel_length)
    
    x_fft = np.fft.fft(kernel)
    sigma_x = np.abs(x_fft)**2
    return np.abs(np.sum(kernel**2)-np.mean(sigma_x))

for n_pulses in range(10,100):
    kernel_length = 200; padding_size = kernel_length
    assert check_equiv(n_pulses, kernel_length, padding_size, 'binary') < 1e-10
    assert check_equiv(n_pulses, kernel_length, padding_size, 'uniform') < 1e-10
    
print("All assertions passed")

All assertions passed


# Upper Bound on SNR
Proposition: For $\sum_{i=1}^n v_i = N$, we have that $f\geq\sqrt{N}$, i.e. the DNF grows at a rate of at least $\sqrt{N}$.

Corrolary: For any fixed $N$, the best achievable SNR is
$$SNR \leq \frac{\bar{i}_0 \sqrt{N}}{\sqrt{\sigma_{\text{gray}}^2 + \sigma_d^2 N}}\:.$$

*Proof*: Using the previous two lemmas, we have that $ f^2 \geq \frac{N^2}{\sum_{j=1}^n v_j^2}$. Next, notice the fact that
$$\max_{v\in[0,1]^n} \sum_{j=1}^nv_j^2~:~ \sum_{j=1}^n v_j=N$$
is achieved for binary $v$ and has the maximum value $N$ (for integer valued $N$). This gives $f^2\geq N$ which yields the result.
 

### Importance of $\sigma_\text{gray}$

The above corrollary shows an *upper bound* on SNR that increases with ${N}$.

However, if $\sigma_{\text{gray}}^2\ll \sigma_d^2 $, the SNR will not increase with $N$, and in fact its maximum value,
$$SNR \leq \frac{\bar{i}_0 \sqrt{N}}{\sqrt{\sigma_d^2 N}} = \frac{\bar{i}_0}{\sigma_d}$$
is achieved by strobed illumination (i.e. $N=1$). In other words, when $\sigma_\text{gray}$ is small, strobed will be optimal.

If $\sigma_\text{gray}$ is larger, then the upper bound suggests that the SNR will improve with ${N}$. However, since it is only an upper bound, it is not sufficient. To actually show that there are settings under which SNR improves with ${N}$, we need to show a *lower bound* that increases with $N$.


# Upper Bound on DNF

Lemma: We have the bound
$$  f^2 \leq \frac{1}{m} + \frac{m-1}{m}\frac{N^2}{\sigma_m^2}\:.$$

*Proof*: Starting with $f^2 = \frac{1}{m} \sum_{i=1}^m \frac{\sigma_1^2}{\sigma_i^2}$, note that $\frac{\sigma_1^2}{\sigma_i^2} \leq \frac{\sigma_1^2}{\sigma_m^2}$ for all $i$.
Plugging in this bound for $i=1,...,m-1$, we see that
$$f^2 \leq \frac{1}{m} + \frac{1}{m}\frac{N^2}{\sigma_m^2}\cdot (m-1)\:,$$
where we use that $\sigma_1$ is the DC component of the signal.

# Thoughts on Lower Bound on SNR

To make statement about a lower bound on SNR, we need to have an upper bound on $f$, which requires a lower bound on $\sigma_m^2$ (as per the above lemma). Of course, one lower bound would be zero, but this gives a vacuous result.

If we could show that $\sigma_m^2\geq c N^{2p}$ for some $0<p<1$, we could show that $f\leq CN^{1-p}$ (for some $C$), which would mean  $SNR \geq \frac{\bar{i}_0 N^{p}}{C \sqrt{\sigma_{\text{gray}}^2 + \sigma_d^2 N}}$. This lower bound shows that there is a regime (where $N$ is small enough) in which the SNR increases with $N$.

